## Improve the model by stacking multiple models into a larger model

In [ ]:
# Install the OpenSlide C library and Python bindings
# After installing these libraries, use `Runtime -> restart and run all` on the menu
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 0s (222 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2

In [ ]:
%matplotlib inline
import cv2
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray

In [ ]:
# Read a region from the slide
# Return a numpy RBG array
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

In [ ]:
#Find tissue 
def find_tissue_pixels(image, intensity=0.8):
    im_gray = rgb2gray(image)
    assert im_gray.shape == (image.shape[0], image.shape[1])
    indices = np.where(im_gray <= intensity)
    return list(zip(indices[0], indices[1]))

In [ ]:
#Highlight tissue by coloring it red
def apply_mask(im, mask, color=(255,0,0)):
    masked = np.zeros((im.shape[0],im.shape[1],im.shape[2]))
    for x,y in mask: masked[x][y] = color
    return masked

In [ ]:
def preprocessing(slide_path,tumor_mask_path,patch_pixel,zoom_level):

  #Create different folders with zoom level
  zoom = str(zoom_level)
  colab_root = '/content/'
  tumor_f = os.path.join(colab_root,zoom+'_tumor/')
  no_tumor_f = os.path.join(colab_root,zoom+'_no_tumor/')
  tumor_mask_f = os.path.join(colab_root,zoom+'_tumor_mask/') 
  no_tumor_mask_f = os.path.join(colab_root,zoom+'_no_tumor_mask/') 

  if os.path.exists(tumor_f) or os.path.exists(no_tumor_f) or os.path.exists(no_tumor_mask_f) or os.path.exists(tumor_mask_f):
    print("Some folders already exist")
  else:
    os.mkdir(tumor_f)
    os.mkdir(no_tumor_f)
    os.mkdir(tumor_mask_f)
    os.mkdir(no_tumor_mask_f) 

  #get image file name from specific slide path for future use
  imagename = slide_path.split('.')[0].strip()

  #open slide
  slide = open_slide(slide_path)
  tumor_mask = open_slide(tumor_mask_path)

  #get slide's size
  slide_width = slide.level_dimensions[zoom_level][0]
  slide_height = slide.level_dimensions[zoom_level][1]
  print('Slide path: '+ slide_path +' Slide Width:',slide_width, ' Slide height:', slide_height)

  #read entire slide and mask at given zoom level
  slide = read_slide(slide, x=0, y=0, level=zoom_level, width=slide_width,height=slide_height)
  tumor_mask = read_slide(tumor_mask, x=0, y=0, level=zoom_level, width=slide_width,height=slide_height)

  # Note: the program provided by the dataset authors generates a mask with R,G,B channels.
  # The mask info we need is in the first channel only.
  tumor_mask = tumor_mask[:,:,0]

  # Improve efficiency by ignoring non-tissue areas of the slide.
  # We'll find these by looking for all red regions.
  tissue_pixels = find_tissue_pixels(slide)
  tissue_regions = apply_mask(slide, tissue_pixels)

  #Now we need to divide patches into corresponding folders
  #First, given size of patch, get the number of patches
  x,y = slide.shape[0], slide.shape[1]
  num_xaxis, num_yaxis = int(np.ceil(x/patch_pixel)), int(np.ceil(y/patch_pixel))
  print('Total number of pathches for this slide (including non-tissue region):', 
        int(num_xaxis*num_yaxis))

  #Count tumor/ no tumor patch
  count_t = 0
  count_no_t = 0

  #Go through every patch and find whether tumors exist
  for i in range(num_xaxis):
    for j in range(num_yaxis):
      #Find patch's x,y corrdinates
      #There are also some patches whose pixels < patch_pixel as we cannot divide slide evenly
      if i == num_xaxis -1:
        slide_x_end = x
        patch_x_end = x-(num_xaxis-1)*patch_pixel
      else:
        slide_x_end = (i+1)*patch_pixel
        patch_x_end = patch_pixel
 
      if j == num_yaxis -1:
        slide_y_end = y
        patch_y_end = y-(num_yaxis-1)*patch_pixel
      else:
        slide_y_end = (j+1)*patch_pixel
        patch_y_end = patch_pixel
      
      #Get patch and its mask from tissue region and check whether tissue exists
      #If not exist, not save the patch and its mask
      check_patch_tissue=np.zeros((patch_pixel,patch_pixel,3))
      patch_img = np.zeros((patch_pixel,patch_pixel,3))
      patch_mask_img = np.zeros((patch_pixel,patch_pixel))
      
      check_patch_tissue[0:patch_x_end,0:patch_y_end,:] = tissue_regions[i*patch_pixel:slide_x_end, j*patch_pixel:slide_y_end,:]

      if np.mean(check_patch_tissue) > float(0): # there is tissue
        #Get patch and its mask
        patch_img[0:patch_x_end,0:patch_y_end,:]=slide[i*patch_pixel:slide_x_end, j*patch_pixel:slide_y_end,:]
        patch_mask_img[0:patch_x_end,0:patch_y_end]=tumor_mask[i*patch_pixel:slide_x_end, j*patch_pixel:slide_y_end]

        #Check wether there is tumor inside the patch through its mask image      
        if np.mean(patch_mask_img)> float(0): #there is tumor
          count_t+=1
          patch_filename = tumor_f+str(imagename)+'_patch_'+str(i*num_yaxis+j)+'.jpg'
          patch_mask_filename = tumor_mask_f+str(imagename)+'_patch_'+str(i*num_yaxis+j)+'.jpg'

        else: #no tumor
          count_no_t+=1
          patch_filename = no_tumor_f+str(imagename)+'_patch_'+str(i*num_yaxis+j)+'.jpg'
          patch_mask_filename = no_tumor_mask_f+str(imagename)+'_patch_'+str(i*num_yaxis+j)+'.jpg'  

        #save patches with corresponding file name
        cv2.imwrite(patch_filename,patch_img)  # put the patch img under the patch_filename path 
        cv2.imwrite(patch_mask_filename,patch_mask_img)  
  print('Number of tumor patches:',count_t,' Number of no tumor patches:',count_no_t)
  

### Load Data
The slides we randomly chose are 'tumor_091', 'tumor_101' and 'tumor_110'.

In [ ]:
# Remove previously created files with zoom level 3

!rm -rf tumor
!rm -rf tumor_mask
!rm -rf no_tumor
!rm -rf no_tumor_mask


In [ ]:
# Download example slides and tumor masks

slide_path1 = 'tumor_091.tif' 
tumor_mask_path1 = 'tumor_091_mask.tif' 

slide_url1 = 'https://storage.googleapis.com/applied-dl/%s' % slide_path1
mask_url1 = 'https://storage.googleapis.com/applied-dl/%s' % tumor_mask_path1

# Download the whole slide image
if not os.path.exists(slide_path1):
  !curl -O $slide_url1

# Download the tumor mask
if not os.path.exists(tumor_mask_path1):
  !curl -O $mask_url1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  521M  100  521M    0     0   246M      0  0:00:02  0:00:02 --:--:--  246M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.6M  100 14.6M    0     0  57.9M      0 --:--:-- --:--:-- --:--:-- 57.7M


In [ ]:
# second slide
slide_path2 = 'tumor_110.tif' 
tumor_mask_path2 = 'tumor_110_mask.tif' 

slide_url2 = 'https://storage.googleapis.com/adl_final4955/%s' % slide_path2
mask_url2 = 'https://storage.googleapis.com/adl_final4955/%s' % tumor_mask_path2

if not os.path.exists(slide_path2):
  !curl -O $slide_url2

if not os.path.exists(tumor_mask_path2):
  !curl -O $mask_url2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1416M  100 1416M    0     0  86.6M      0  0:00:16  0:00:16 --:--:-- 61.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.3M  100 31.3M    0     0  32.6M      0 --:--:-- --:--:-- --:--:-- 32.6M


In [ ]:
# third slide
slide_path3 = 'tumor_101.tif' 
tumor_mask_path3 = 'tumor_101_mask.tif' 

slide_url3 = 'https://storage.googleapis.com/adl_final4955/%s' % slide_path3
mask_url3 = 'https://storage.googleapis.com/adl_final4955/%s' % tumor_mask_path3

if not os.path.exists(slide_path3):
  !curl -O $slide_url3

if not os.path.exists(tumor_mask_path3):
  !curl -O $mask_url3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1392M  100 1392M    0     0  37.5M      0  0:00:37  0:00:37 --:--:-- 46.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.9M  100 43.9M    0     0  37.8M      0  0:00:01  0:00:01 --:--:-- 37.9M


In order to fit model well, we choose an patch size 299*299. 

Zoom Level 3

In [ ]:
preprocessing(slide_path1,tumor_mask_path1,299,3)

Slide path: tumor_091.tif Slide Width: 7680  Slide height: 6720
Total number of pathches for this slide (including non-tissue region): 598
Number of tumor patches: 27  Number of no tumor patches: 561


In [ ]:
preprocessing(slide_path2,tumor_mask_path2,299,3)

Some folders already exist
Slide path: tumor_110.tif Slide Width: 11776  Slide height: 8960
Total number of pathches for this slide (including non-tissue region): 1200
Number of tumor patches: 275  Number of no tumor patches: 841


In [ ]:
preprocessing(slide_path3,tumor_mask_path3,299,3)

Some folders already exist
Slide path: tumor_101.tif Slide Width: 17408  Slide height: 8960
Total number of pathches for this slide (including non-tissue region): 1770
Number of tumor patches: 82  Number of no tumor patches: 1634


In [ ]:
no_tumor_f_len = len(os.listdir('/content/3_no_tumor'))
tumor_f_len = len(os.listdir('/content/3_tumor'))
print('Patch without tumor & zoom level 3:', no_tumor_f_len,' Patch with tumor & zoom level 3:',tumor_f_len)

Patch without tumor & zoom level 3: 3036  Patch with tumor & zoom level 3: 384


Zoom Level 4

In [ ]:
preprocessing(slide_path1,tumor_mask_path1,299,4)

Slide path: tumor_091.tif Slide Width: 3840  Slide height: 3360
Total number of pathches for this slide (including non-tissue region): 156
Number of tumor patches: 12  Number of no tumor patches: 143


In [ ]:
preprocessing(slide_path2,tumor_mask_path2,299,4)

Some folders already exist
Slide path: tumor_110.tif Slide Width: 5888  Slide height: 4480
Total number of pathches for this slide (including non-tissue region): 300
Number of tumor patches: 88  Number of no tumor patches: 191


In [ ]:
preprocessing(slide_path3,tumor_mask_path3,299,4)

Some folders already exist
Slide path: tumor_101.tif Slide Width: 8704  Slide height: 4480
Total number of pathches for this slide (including non-tissue region): 450
Number of tumor patches: 40  Number of no tumor patches: 398


In [ ]:
no_tumor_f_len = len(os.listdir('/content/4_no_tumor'))
tumor_f_len = len(os.listdir('/content/4_tumor'))
print('Patch without tumor & zoom level 4:', no_tumor_f_len,' Patch with tumor & zoom level 4:',tumor_f_len)

Patch without tumor & zoom level 4: 732  Patch with tumor & zoom level 4: 140


Zoom Level 5

In [ ]:
preprocessing(slide_path1,tumor_mask_path1,299,5)

Slide path: tumor_091.tif Slide Width: 1920  Slide height: 1680
Total number of pathches for this slide (including non-tissue region): 42
Number of tumor patches: 6  Number of no tumor patches: 36


In [ ]:
preprocessing(slide_path2,tumor_mask_path2,299,5)

Some folders already exist
Slide path: tumor_110.tif Slide Width: 2944  Slide height: 2240
Total number of pathches for this slide (including non-tissue region): 80
Number of tumor patches: 34  Number of no tumor patches: 44


In [ ]:
preprocessing(slide_path3,tumor_mask_path3,299,5)

Some folders already exist
Slide path: tumor_101.tif Slide Width: 4352  Slide height: 2240
Total number of pathches for this slide (including non-tissue region): 120
Number of tumor patches: 18  Number of no tumor patches: 101


In [ ]:
no_tumor_f_len = len(os.listdir('/content/5_no_tumor'))
tumor_f_len = len(os.listdir('/content/5_tumor'))
print('Patch without tumor & zoom level 5:', no_tumor_f_len,' Patch with tumor & zoom level 5:',tumor_f_len)

Patch without tumor & zoom level 5: 181  Patch with tumor & zoom level 5: 58


### Create Training, Validation and Test Sets

We can see the result above that the number of patches without tumor is quite larger than that of patches with tumor. This will result an very imbalanced dataset. Hence, we have decided to create a balanced dataset with all tumor patches we have and no tumor patches with similar size. Notice, the size of the dataset will not be large since we have a small number of tumor patches. This may have some impacts on validation accuracy later since we do not have much data to train.


When creating training,validation and test sets, we follow a ratio of 4:1:1.

In [ ]:
#Create train, val and test directory
#zoom level 3
train_z3 = '/content/train_z3'
val_z3 = '/content/val_z3'
test_z3 = '/content/test_z3'

os.mkdir(train_z3)
os.mkdir(val_z3)
os.mkdir(test_z3)

train_tumor3 = os.path.join(train_z3,'tumor')
train_no_tumor3 = os.path.join(train_z3,'no_tumor')
os.mkdir(train_tumor3)
os.mkdir(train_no_tumor3)

val_tumor3 = os.path.join(val_z3,'tumor')
val_no_tumor3 = os.path.join(val_z3,'no_tumor')
os.mkdir(val_tumor3)
os.mkdir(val_no_tumor3)

test_tumor3 = os.path.join(test_z3,'tumor')
test_no_tumor3 = os.path.join(test_z3,'no_tumor')
os.mkdir(test_tumor3)
os.mkdir(test_no_tumor3)

#zoom level 4
train_z4 = '/content/train_z4'
val_z4 = '/content/val_z4'
test_z4 = '/content/test_z4'

os.mkdir(train_z4)
os.mkdir(val_z4)
os.mkdir(test_z4)

train_tumor4 = os.path.join(train_z4,'tumor')
train_no_tumor4 = os.path.join(train_z4,'no_tumor')
os.mkdir(train_tumor4)
os.mkdir(train_no_tumor4)

val_tumor4 = os.path.join(val_z4,'tumor')
val_no_tumor4 = os.path.join(val_z4,'no_tumor')
os.mkdir(val_tumor4)
os.mkdir(val_no_tumor4)

test_tumor4 = os.path.join(test_z4,'tumor')
test_no_tumor4 = os.path.join(test_z4,'no_tumor')
os.mkdir(test_tumor4)
os.mkdir(test_no_tumor4)

#zoom level 5
train_z5 = '/content/train_z5'
val_z5 = '/content/val_z5'
test_z5 = '/content/test_z5'

os.mkdir(train_z5)
os.mkdir(val_z5)
os.mkdir(test_z5)

train_tumor5 = os.path.join(train_z5,'tumor')
train_no_tumor5 = os.path.join(train_z5,'no_tumor')
os.mkdir(train_tumor5)
os.mkdir(train_no_tumor5)

val_tumor5 = os.path.join(val_z5,'tumor')
val_no_tumor5 = os.path.join(val_z5,'no_tumor')
os.mkdir(val_tumor5)
os.mkdir(val_no_tumor5)

test_tumor5 = os.path.join(test_z5,'tumor')
test_no_tumor5 = os.path.join(test_z5,'no_tumor')
os.mkdir(test_tumor5)
os.mkdir(test_no_tumor5)

In [ ]:
#shuffle tumor and no_tumor folders
import random
#zoom level 3
tumor_img3 = os.listdir('/content/3_tumor')
no_tumor_img3 = os.listdir('/content/3_no_tumor')
random.shuffle(tumor_img3)
random.shuffle(no_tumor_img3)
tumor_len3 =len(tumor_img3)
no_tumor_len3 = len(no_tumor_img3)
print('Zoom level 3:')
print('Tumor:',tumor_len3, ' No Tumor:', no_tumor_len3)

#zoom level 4
tumor_img4 = os.listdir('/content/4_tumor')
no_tumor_img4 = os.listdir('/content/4_no_tumor')
random.shuffle(tumor_img4)
random.shuffle(no_tumor_img4)
tumor_len4 =len(tumor_img4)
no_tumor_len4 = len(no_tumor_img4)
print('Zoom level 4:')
print('Tumor:',tumor_len4, ' No Tumor:', no_tumor_len4)

#zoom level 5
tumor_img5 = os.listdir('/content/5_tumor')
no_tumor_img5 = os.listdir('/content/5_no_tumor')
random.shuffle(tumor_img5)
random.shuffle(no_tumor_img5)
tumor_len5 =len(tumor_img5)
no_tumor_len5 = len(no_tumor_img5)
print('Zoom level 5:')
print('Tumor:',tumor_len5, ' No Tumor:', no_tumor_len5)

Zoom level 3:
Tumor: 384  No Tumor: 3036
Zoom level 4:
Tumor: 140  No Tumor: 732
Zoom level 5:
Tumor: 58  No Tumor: 181


As we only have 384 patches with tumor, we will use 384 patches without tumor to keep balance. We divide those tumor patches and no-tumor pathces into training, validation and testing ratio of 4:1:1, and put them into respective folders. 

In [ ]:
import shutil
#Zoom level 3
#Training-Tumor
for im in tumor_img3[0:int(tumor_len3*4/6)]:
  shutil.copy(os.path.join('/content/3_tumor',im),train_tumor3)

#Validation-Tumor
for im in tumor_img3[int(tumor_len3*4/6):int(tumor_len3*5/6)]:
  shutil.copy(os.path.join('/content/3_tumor',im),val_tumor3)

# Test-Tumor
for im in tumor_img3[int(tumor_len3*5/6):]:
  shutil.copy(os.path.join('/content/3_tumor',im),test_tumor3)


#Training-NoTumor
for im in no_tumor_img3[0:int(tumor_len3*4/6)]:
  shutil.copy(os.path.join('/content/3_no_tumor',im),train_no_tumor3)

#Validation-NoTumor
for im in no_tumor_img3[int(tumor_len3*4/6):int(tumor_len3*5/6)]:
  shutil.copy(os.path.join('/content/3_no_tumor',im),val_no_tumor3)

#Test-NoTumor
for im in no_tumor_img3[int(tumor_len3*5/6):tumor_len3]:
  shutil.copy(os.path.join('/content/3_no_tumor',im),test_no_tumor3)

#Zoom level 4
#Training-Tumor
for im in tumor_img4[0:int(tumor_len4*4/6)]:
  shutil.copy(os.path.join('/content/4_tumor',im),train_tumor4)

#Validation-Tumor
for im in tumor_img4[int(tumor_len4*4/6):int(tumor_len4*5/6)]:
  shutil.copy(os.path.join('/content/4_tumor',im),val_tumor4)

# Test-Tumor
for im in tumor_img4[int(tumor_len4*5/6):]:
  shutil.copy(os.path.join('/content/4_tumor',im),test_tumor4)


#Training-NoTumor
for im in no_tumor_img4[0:int(tumor_len4*4/6)]:
  shutil.copy(os.path.join('/content/4_no_tumor',im),train_no_tumor4)

#Validation-NoTumor
for im in no_tumor_img4[int(tumor_len4*4/6):int(tumor_len4*5/6)]:
  shutil.copy(os.path.join('/content/4_no_tumor',im),val_no_tumor4)

#Test-NoTumor
for im in no_tumor_img4[int(tumor_len4*5/6):tumor_len4]:
  shutil.copy(os.path.join('/content/4_no_tumor',im),test_no_tumor4)


#Zoom level 5
#Training-Tumor
for im in tumor_img5[0:int(tumor_len5*4/6)]:
  shutil.copy(os.path.join('/content/5_tumor',im),train_tumor5)

#Validation-Tumor
for im in tumor_img5[int(tumor_len5*4/6):int(tumor_len5*5/6)]:
  shutil.copy(os.path.join('/content/5_tumor',im),val_tumor5)

# Test-Tumor
for im in tumor_img5[int(tumor_len5*5/6):]:
  shutil.copy(os.path.join('/content/5_tumor',im),test_tumor5)


#Training-NoTumor
for im in no_tumor_img5[0:int(tumor_len5*4/6)]:
  shutil.copy(os.path.join('/content/5_no_tumor',im),train_no_tumor5)

#Validation-NoTumor
for im in no_tumor_img5[int(tumor_len5*4/6):int(tumor_len5*5/6)]:
  shutil.copy(os.path.join('/content/5_no_tumor',im),val_no_tumor5)

#Test-NoTumor
for im in no_tumor_img5[int(tumor_len5*5/6):tumor_len5]:
  shutil.copy(os.path.join('/content/5_no_tumor',im),test_no_tumor5)


In [ ]:
print(len(os.listdir('/content/train_z3/tumor')))
print(len(os.listdir('/content/train_z3/no_tumor')))
print(len(os.listdir('/content/train_z4/tumor')))
print(len(os.listdir('/content/train_z4/no_tumor')))
print(len(os.listdir('/content/train_z5/tumor')))
print(len(os.listdir('/content/train_z5/no_tumor')))

256
256
93
93
38
38


### Create Datasets

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator3 = train_datagen.flow_from_directory(
        train_z3, # a path as string 
        target_size=(299,299), 
        class_mode='binary',
        batch_size=32)

for data_batch3, labels_batch3 in train_generator3:
    print('data batch shape in zoom level 3:', data_batch3.shape)
    print('labels batch shape in zoom level 3:', labels_batch3.shape)  
    break

Found 512 images belonging to 2 classes.
data batch shape in zoom level 3: (32, 299, 299, 3)
labels batch shape in zoom level 3: (32,)


### Training Model
#### Stacking three InceptionV3 models, each process images with a zoom  level of 3, 4 and 5 respectively. Then add a concatenating layer and a few layers on top. 


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, Sequential

# First InceptionV3 model 
input_1 = Input(shape=(299, 299, 3))
base_model_1 = Sequential()

ince_1 = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(299,299, 3)
)
base_model_1.add(ince_1)
base_model_1.trainable = False 
encoded_input_1 = base_model_1(input_1)

# Second InceptionV3 model
input_2 = Input(shape=(299, 299, 3))
base_model_2 = Sequential()
ince_2 = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(299,299, 3)
)
base_model_2.add(ince_2)
base_model_2.trainable = False 
encoded_input_2 = base_model_2(input_2)

# Third InceptionV3 model
input_3 = Input(shape=(299, 299, 3))
base_model_3 = Sequential()
ince_3 = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(299,299, 3)
)
base_model_3.add(ince_3)
base_model_3.trainable = False 
encoded_input_3 = base_model_3(input_3)

# Let's concatenate 3 base models:
merged = tf.keras.layers.concatenate([encoded_input_1,encoded_input_2,encoded_input_3])

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(merged)

dense = tf.keras.layers.Dense(30, activation='relu')(global_average_layer)

# Next, add a binary classifier on top
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

# This is our final model:
final_model = Model(inputs=[input_1,input_2,input_3], outputs=output)


final_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Create a function to read in intputs from 3 folders with different zoom levels  

def multiple_generators(gen, X1, X2, X3):
    genX1 = gen.flow_from_directory(directory = X1, target_size=(299,299), class_mode='binary',batch_size = 2)
    genX2 = gen.flow_from_directory(directory = X2, target_size=(299,299), class_mode='binary',batch_size = 2)
    genX3 = gen.flow_from_directory(directory = X3, target_size=(299,299), class_mode='binary',batch_size = 2)
    while True:
        X1i = genX1.next() # get the next batch
        X2i = genX2.next()
        X3i = genX3.next()
        yield [X1i[0], X2i[0], X3i[0]], X1i[1] # [0] get batch's images, and [1] get labels 

In [ ]:
train_multiple_generator = multiple_generators(train_datagen, train_z3, train_z4, train_z5)

In [ ]:
history = final_model.fit( 
      train_multiple_generator,
      epochs=1)

  65078/Unknown - 3686s 57ms/step - loss: 0.0081 - accuracy: 0.9975

In [ ]:
# train_generator3.next()[0].shape, len(train_generator3.next()[1]) # ((32, 299, 299, 3), 32)

# len(validation_generator4.next()[1]) # 32